In [ ]:
pip install legacy-cgi googletrans==3.1.0a0

In [48]:
import os
import pandas as pd
from googletrans import Translator

In [49]:
os.listdir()

['.config',
 'donneesouvertes-interventions-sim-2005-2014.csv',
 'incidents-clean.csv',
 'type-interventions-descriptions20161122.csv',
 'donneesouvertes-interventions-sim_2015_2022.csv',
 'donneesouvertes-interventions-sim.csv',
 'donneesouvertes-interventions-sim2020.csv',
 'sample_data']

In [50]:
df_2005_2014 = pd.read_csv('donneesouvertes-interventions-sim-2005-2014.csv')
df_2015_2022 = pd.read_csv('donneesouvertes-interventions-sim_2015_2022.csv').drop(columns=['MTM8_X', 'MTM8_Y'])
df_2020_2023 = pd.read_csv('donneesouvertes-interventions-sim2020.csv').drop(columns=['MTM8_X', 'MTM8_Y'])
df_2024_2025 = pd.read_csv('donneesouvertes-interventions-sim.csv').drop(columns=['MTM8_X', 'MTM8_Y'])

In [51]:
df_2005_2014['CREATION_DATE_TIME'] = pd.to_datetime(df_2005_2014['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')
df_2015_2022['CREATION_DATE_TIME'] = pd.to_datetime(df_2015_2022['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')
df_2020_2023['CREATION_DATE_TIME'] = pd.to_datetime(df_2020_2023['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')
df_2024_2025['CREATION_DATE_TIME'] = pd.to_datetime(df_2024_2025['CREATION_DATE_TIME'], format='ISO8601').dt.strftime('%Y-%m-%d')

In [52]:
df_incidents = pd.concat([df_2005_2014, df_2015_2022, df_2020_2023, df_2024_2025], ignore_index=True)
df_incidents.rename(columns={'CREATION_DATE_TIME': 'CREATION_DATE'}, inplace=True)
df_incidents.drop_duplicates(subset=['INCIDENT_NBR', 'CREATION_DATE'], inplace=True)
df_incidents.shape

(2050879, 11)

In [53]:
df_incidents.sort_values(by=['CREATION_DATE','INCIDENT_NBR'], inplace=True)
df_incidents.head()

,INCIDENT_NBR,CREATION_DATE,INCIDENT_TYPE_DESC,DESCRIPTION_GROUPE,CASERNE,NOM_VILLE,NOM_ARROND,DIVISION,LATITUDE,LONGITUDE,NOMBRE_UNITES
0,1,2005-01-01,Inondation,Sans incendie,18,Montréal-Nord,Montréal-Nord,21,45.620274,-73.619956,1.0
1,2,2005-01-01,Alarme privé ou locale,Alarmes-incendies,10,Montréal,Ville-Marie / Parc Jean-Drapeau / Centre-Sud,18,45.494087,-73.582587,7.0
2,3,2005-01-01,Alarme privé ou locale,Alarmes-incendies,72,Saint-Laurent,St-Laurent,13,45.484406,-73.693038,6.0
3,4,2005-01-01,Inondation,Sans incendie,13,Montréal,Mercier / Hochelaga-Maisonneuve,23,45.541383,-73.545944,1.0
4,5,2005-01-01,Inondation,Sans incendie,18,Montréal-Nord,Montréal-Nord,21,45.611304,-73.632440,1.0


In [54]:
df_incidents['DESCRIPTION_GROUPE'].unique()

array(['Sans incendie', 'Alarmes-incendies', 'Autres incendies',
       'Fausses alertes/annulations', 'Premier répondant',
       'Incendies de bâtiments', 'SANS FEU', '1-REPOND', 'AUTREFEU',
       'INCENDIE', 'FAU-ALER', 'NOUVEAU', nan], dtype=object)

In [55]:
description_groups_translated = {
    "1-REPOND": "First response",
    "Premier répondant": "First response",
    "Alarmes-incendies": "Fire alarm",
    "SANS FEU": "No fire",
    "Sans incendie": "No fire",
    "Autres incendies": "Other fires",
    "AUTREFEU": "Other fires",
    "Incendies de bâtiments": "Building fire",
    "Fausses alertes/annulations": "False alarm/cancellation",
    "INCENDIE": "Building fire",
    "FAU-ALER": "False alarm/cancellation",
    "NOUVEAU": "New"
}
df_incidents['DESC_GROUP_ENGLISH'] = df_incidents['DESCRIPTION_GROUPE'].map(description_groups_translated)
df_incidents.head()

,INCIDENT_NBR,CREATION_DATE,INCIDENT_TYPE_DESC,DESCRIPTION_GROUPE,CASERNE,NOM_VILLE,NOM_ARROND,DIVISION,LATITUDE,LONGITUDE,NOMBRE_UNITES,DESC_GROUP_ENGLISH
0,1,2005-01-01,Inondation,Sans incendie,18,Montréal-Nord,Montréal-Nord,21,45.620274,-73.619956,1.0,No fire
1,2,2005-01-01,Alarme privé ou locale,Alarmes-incendies,10,Montréal,Ville-Marie / Parc Jean-Drapeau / Centre-Sud,18,45.494087,-73.582587,7.0,Fire alarm
2,3,2005-01-01,Alarme privé ou locale,Alarmes-incendies,72,Saint-Laurent,St-Laurent,13,45.484406,-73.693038,6.0,Fire alarm
3,4,2005-01-01,Inondation,Sans incendie,13,Montréal,Mercier / Hochelaga-Maisonneuve,23,45.541383,-73.545944,1.0,No fire
4,5,2005-01-01,Inondation,Sans incendie,18,Montréal-Nord,Montréal-Nord,21,45.611304,-73.632440,1.0,No fire


In [56]:
"""
translator = Translator()
df_type_descriptions = pd.read_csv('type-interventions-descriptions20161122.csv')
df_type_descriptions.rename(columns={'INCIDENT_TYPE_DESCRIPTION': 'INCIDENT_TYPE_DESC', 'Description': 'DESCRIPTION'}, inplace=True)
df_type_descriptions.drop(
    df_type_descriptions[df_type_descriptions['DESCRIPTION'] == "Type qui n'est plus utilisé à la suite de la mise à jour de l'algorithme en 2016"].index,
    inplace=True
)
df_type_descriptions['DESC_ENGLISH'] = df_type_descriptions['DESCRIPTION'].apply(translator.translate, src='fr', dest='en').apply(getattr, args=('text',))
df_type_descriptions.head()
"""

'\ntranslator = Translator()\ndf_type_descriptions = pd.read_csv(\'type-interventions-descriptions20161122.csv\')\ndf_type_descriptions.rename(columns={\'INCIDENT_TYPE_DESCRIPTION\': \'INCIDENT_TYPE_DESC\', \'Description\': \'DESCRIPTION\'}, inplace=True)\ndf_type_descriptions.drop(\n    df_type_descriptions[df_type_descriptions[\'DESCRIPTION\'] == "Type qui n\'est plus utilisé à la suite de la mise à jour de l\'algorithme en 2016"].index,\n    inplace=True\n)\ndf_type_descriptions[\'DESC_ENGLISH\'] = df_type_descriptions[\'DESCRIPTION\'].apply(translator.translate, src=\'fr\', dest=\'en\').apply(getattr, args=(\'text\',))\ndf_type_descriptions.head()\n'

In [57]:
"""
df_merged = pd.merge(df_incidents, df_type_descriptions, on='INCIDENT_TYPE_DESC', how='left')
df_merged = df_merged.iloc[:, [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13]]
print(df_merged.shape)
df_merged.head()
"""

"\ndf_merged = pd.merge(df_incidents, df_type_descriptions, on='INCIDENT_TYPE_DESC', how='left')\ndf_merged = df_merged.iloc[:, [1, 0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 11, 13]]\nprint(df_merged.shape)\ndf_merged.head()\n"

In [58]:
df_na = df_incidents[df_incidents.isna().any(axis=1) == True]
df_na.shape

(1784, 12)

In [59]:
df_na.isna().sum()

,0
INCIDENT_NBR,0
CREATION_DATE,0
INCIDENT_TYPE_DESC,0
DESCRIPTION_GROUPE,1034
CASERNE,0
NOM_VILLE,0
NOM_ARROND,0
DIVISION,0
LATITUDE,1
LONGITUDE,1


In [60]:
df_incidents.dropna(axis=0, inplace=True)
df_incidents.shape

(2049095, 12)

In [61]:
df_incidents['DESC_GROUP_ENGLISH'].value_counts()

,count
DESC_GROUP_ENGLISH,
First response,1227941
No fire,434764
Fire alarm,286956
Other fires,67051
Building fire,26850
False alarm/cancellation,5526
New,7


In [62]:
df_incidents_clean = df_incidents[df_incidents['DESC_GROUP_ENGLISH'].isin(['Fire', 'Building fire', 'Other fires'])]
df_incidents_clean.shape
df_incidents_clean['DESC_GROUP_ENGLISH'].value_counts()

,count
DESC_GROUP_ENGLISH,
Other fires,67051
Building fire,26850


In [63]:
df_incidents_clean['CASERNE'].unique()

array([22, 47, 43, 15, 41, 14, 76,  9, 53, 26, 21, 18, 54, 33, 20, 23, 27,
       35, 17, 49, 44, 29, 72, 65, 40, 73, 30, 38, 19, 48,  4, 52, 42, 66,
       79, 64, 55, 34,  5, 37, 57, 39, 13, 16, 10, 45, 67, 50, 61, 46, 25,
       31, 63, 28, 71,  8, 74, 51,  3, 78, 75, 62, 58, 77, 56, 59, 32])

In [64]:
df_incidents_clean['NOM_VILLE'].unique()

array(['Saint-Léonard', 'Montréal', 'Westmount', 'Beaconsfield',
       'Montréal-Nord', 'Kirkland', 'Saint-Laurent', 'Lasalle',
       "Baie d'Urfé", 'Verdun / Ïle-des-Soeurs', 'Hampstead', 'Lachine',
       'Pointe-Claire', 'Pierrefonds ', 'Dollard-des-Ormeaux', 'Dorval',
       'Anjou', 'Montréal-Est', 'Mont-Royal', 'Ste-Anne-de-Bellevue',
       'Côte St-Luc', 'Outremont', 'Ile-Bizard', 'Roxboro',
       'Montréal-Ouest', 'Senneville', 'Ste-Geneviève', 'Côte-Saint-Luc',
       'Sainte-Anne-de-Bellevue', "Baie-d'Urfé", "L'Île-Dorval"],
      dtype=object)

In [65]:
df_incidents_clean['NOM_ARROND'].unique()

array(['St-Léonard', 'Rosemont / Petite-Patrie',
       'Ahuntsic / Cartierville', 'Sud-Ouest',
       'Villeray / St-Michel / Parc Extension',
       'Rivière-des-Prairies / P-A-T/Mtl-Est',
       'Côte-des-Neiges / Notre-Dame-de-Grâce',
       "Beaconsfield / Baie d'Urfé", 'Plateau Mont-Royal',
       'Montréal-Nord', 'Kirkland', 'Westmount',
       'Ville-Marie / Parc Jean-Drapeau / Centre-Sud', 'St-Laurent',
       'Lasalle', 'Mercier / Hochelaga-Maisonneuve',
       'Verdun / Ïle-des-Soeurs', 'Côte St-Luc / Hampstead / Mtl-Ouest',
       'Lachine', 'Pointe-Claire', 'Pierrefonds / Senneville',
       'Dollard-des-Ormeaux / Roxboro', 'Dorval / Ile Dorval', 'Anjou',
       'Mont-Royal', 'Ile-Bizard / Ste-Geneviève / Ste-A-de-B',
       'Outremont', 'Indéterminé',
       'Rivière-des-Prairies / Pointe-aux-Trembles', 'Ville-Marie',
       'Saint-Laurent', 'Pierrefonds / Roxboro', 'Verdun',
       'Saint-Léonard', "L'Ile-Bizard / Ste-Geneviève",
       'Villeray-Saint-Michel-Parc-Extens

In [66]:
df_incidents_clean['DIVISION'].unique()

array([20, 22, 21, 15, 25, 17, 12, 18, 13, 14, 23, 11, 10,  8,  9,  7,  3,
        4,  1,  6,  5,  2,  0])

In [67]:
df_incidents_clean['NOMBRE_UNITES'].unique()

array([  4.,   1.,   7.,   8.,   3.,  16.,  10.,   5.,   2.,  18.,   6.,
        12.,  21.,  15.,  14.,  98.,   9.,  32.,  20.,  31.,  19.,  39.,
        42.,  11.,  23.,  51.,  70.,  35.,  46.,  34.,  13.,  38.,  17.,
        40.,  36.,  41.,  29.,  22.,  56.,  64.,  44.,  89.,  28.,  27.,
        62.,  24.,  52.,  37.,  33.,  43., 100.,  30.,  45.,  80.,  71.,
        91.,  50.,  48.,  79.,  25.,  55.,  57.,  68.,  66.,  49.,  53.,
        77.,  67.,  87.,  92.,  88.,  95.,  26.,  63., 104.,  60.,  73.,
        47.,  90.,  81.,  69., 118.,  61.,  75.,  65., 132.,  54.,  58.,
       107., 135.,  96., 148.,  85., 275.,  78., 123., 119.,  59.,  83.,
        93.,  86., 102.,  76., 117.,  74.,  72.,  94., 163., 126., 120.,
       106., 130., 153.,  82.,  84., 115., 127., 108., 160., 101., 199.,
       103., 244., 105., 172.])

In [68]:
df_incidents_clean.to_csv('incidents-clean.csv', index=False)